# Datenanalyse

Erstmal brauchen wir die Daten und einen groben Überblick:

In [12]:
import pandas as pd
import utils.load_write as lw
import utils.datastructure_operations as dataops
import utils.specific_helpers as sph

df_horse = lw.load_csv("../data/csv/abmap_horse.csv")

measures_per_bone = df_horse.BONEID.value_counts()
sites = df_horse.SITE.value_counts()

print("Es liegen", len(df_horse.index) ,
      "Maße von insgesamt", len(measures_per_bone),
      "Knochen aus" , len(sites),
      "archäologischen Fundstellen vor.")

print("Jeder Datenpunkt hat", len(df_horse.columns), "Attribute:")

dataops.pretty_print(list(df_horse.columns))

Es liegen 3099 Maße von insgesamt 1038 Knochen aus 71 archäologischen Fundstellen vor.
Jeder Datenpunkt hat 17 Attribute:
[ 'BONEID', 'MEASURE', 'MEASTYPE', 'SPECIES', 'ELEMENT', 'SIDE', 'COMMENTS',
  'CONTEXT', 'PERIOD', 'CDATE', 'RANGE', 'PHASE', 'SITECODE', 'SITE', 'COUNTY',
  'GRIDREF', 'REFERENCE']


## Bone Types
Anschließend gruppieren wir die Daten nach den Knochentypen (`ELEMENT`-Attribut), da wir nur innerhalb des gleichen Typs sinnvolle Vergleiche anstellen können:

In [13]:
element_grp = df_horse.groupby(['ELEMENT'])
print("Es existieren Funde zu", element_grp.ngroups, "verschiedenen Knochentypen:")
list(element_grp.groups.keys())

Es existieren Funde zu 14 verschiedenen Knochentypen:


['Astragalus',
 'Calcaneum',
 'Femur',
 'First phalanx',
 'Humerus',
 'Mandible',
 'Metacarpal',
 'Metatarsal',
 'Pelvis',
 'Radius',
 'Scapula',
 'Second phalanx',
 'Tibia',
 'Ulna']

## Datierung
Danach wollen wir wissen, aus welchen zeitlichen Perioden überhaupt Funde vorliegen und ob die Anzahl jeweils ausreicht für eine statistische Auswertung. Dazu betrachten wir die Attribute `PERIOD` und `RANGE`:

In [14]:
periods = dataops.count_amounts_in_column(df_horse, "PERIOD")
print("Anzahl an Funden pro Periode: ")
dataops.pretty_print(periods)

ranges = dataops.count_amounts_in_column(df_horse, "RANGE")
print("Diese", len(periods),
      "Perioden des Datensatzes wurden weiter unterteilt in insgesamt", len(ranges),
      "Zeitintervalle.")

Anzahl an Funden pro Periode: 
{ 'Early - Middle Iron Age': 40,
  'Early Iron Age': 26,
  'Early Medieval': 627,
  'Early Roman': 65,
  'Early Saxon': 2,
  'Iron Age': 43,
  'Late Bronze Age - Early Iron Age': 40,
  'Late Iron Age': 193,
  'Late Iron Age - Early Roman': 163,
  'Late Roman': 289,
  'Late Saxon': 41,
  'Medieval': 790,
  'Medieval - Post Medieval': 39,
  'Mid - Late Iron Age': 16,
  'Mid Roman': 10,
  'Middle Iron Age': 206,
  'Modern': 12,
  'Post Medieval': 90,
  'Roman': 383,
  'Roman - Early Medieval': 14,
  'Saxon': 10}
Diese 21 Perioden des Datensatzes wurden weiter unterteilt in insgesamt 101 Zeitintervalle.


Bei der Betrachtung dieser Listen wird deutlich, dass einige Perioden nur sehr wenige Funde enthalten (2 für 'Early Saxon'), andere hingegen recht viele (790 für 'Medieval').
Die einzelnen Funde wurden zusätzlich zu einer Periode noch jeweils einem etwas genaueren Zeitintervall zugeordnet ('RANGE').
Insgesamt liegen 101 unterschiedliche Zeitintervalle vor, welche den insgesamt 21 Perioden zugeordnet wurden. Um einen groben zeitlichen Überblick über die Daten zu bekommen, sind dies viel zu viele Intervalle.

### Range und Periode zusammenfassen?
Im Folgenden möchte ich versuchen, die Perioden noch etwas gröber zu fassen und Attribute wie 'late' und 'early' auf die "Hauptperiode" zu reduzieren.
Dafür muss ich allerdings sicherstellen, dass sich Zeitspannen, die ich alleine vom Namen her nicht zusammenfassen würde, auch nicht überschneiden.
Außerdem wäre es hilfreich, die gesamte Spannweite der einzelnen Perioden zu kennen.
1. Liste der Perioden der Urgeschichte serialisieren, damit der lange Text nicht dauernd im Code stehen muss.

In [15]:
archaeological_periods = sph.archaeological_periods_dict()
#lw.save_json(archaeological_periods, "../data/json/archaeological-periods.json")

2. Für jede der Perioden alle Range-Attribute finden. Diese bestehen aus einem Tupel $(y_s, y_e)$ mit $y_s =$ Start des Intervalls und $y_e = $ Ende des Intervalls.
Falls es mehrere Ranges für eine Periode gibt, werden diese Tupel zuerst sortiert und zwar aufsteigend nach $y_s$, um einen Überblick zu bekommen.

In [16]:
periods_ranges = lw.load_json("../data/json/archaeological-periods.json")

"""
Get all associated ranges for each period
"""
for period, years in zip(df_horse.PERIOD, df_horse.RANGE):
      if period in periods_ranges:
            if years not in periods_ranges[period]:
                  periods_ranges[period].append(years)
      else:
            periods_ranges[period] = []
            periods_ranges[period].append(years)

"""
The 'RANGE' Attribute is a string, so in order to do anything with these years we have to convert them to int.
"""
for period in periods_ranges:
      for years in periods_ranges[period]:
            periods_ranges[period][periods_ranges[period].index(years)] = dataops.range_to_tuple(years)

      periods_ranges[period].sort(key=lambda x: x[0])

#lw.save_json(periods_ranges, "../data/json/archaeological-periods-w-ranges.json")

3. Anschließend wird für jede Periode aus der Liste ihrer zugehörigen range-Tupel das kleinste $y_s$ und das größe $y_e$ für die maximale range dieser Periode bestimmt, sodass $y_s = p_s =$ Start der Periode und $y_e = p_e =$ Ende der Periode.

In [17]:
"""
From alle the ranges for each period find the earliest and the latest year for the min-max range
"""
period_min_max = {}
periods_ranges = lw.load_json("../data/json/archaeological-periods-w-ranges.json")
for period in periods_ranges:
      period_min_max [period] = (min(periods_ranges[period],key=lambda x : x[0])[0],
                                 max(periods_ranges[period],key=lambda x : x[1])[1])

#lw.save_json(period_min_max, "../data/json/periods-min-max.json")

### Eigene Einteilung der Perioden
Die Datierungen der Funde überlappen sich jedoch zu stark, um eine klare Grenze zu ziehen.
Daher werden die Funde von Hand grob in vier Perioden eingeteilt, s. `../data/json/time-ranges-condensed.json`:
1. Bronze - Iron Age (-1000 - 150)
2. Roman - Saxon (43 - 1139)
3. Medieval (1000 - 1901)
4. Modern (1900 - 2500)

Nun aktualisieren wir für jeden Datenpunkt die jeweilige Datierung (`PERIOD`), sodass wir nur noch mit den vier zuvor festgelegten, gröberen Perioden arbeiten.
Dann zählen wir, aus wie vielen dieser Perioden für jeden Typ Funde vorliegen, denn wenn nur Funde aus einer Periode vorliegen, eignet sich der entsprechende Knochentyp natürlich nicht für einen Vergleich über die Zeit.
Außerdem zählen wir, wie viele verschiedene Knochen jeweils gefunden wurden und wie viele Messungen pro Typ gemacht wurden, denn auch hier gilt: Zu wenige Daten liefern keine aussagekräftigen Ergebnisse.

In [18]:
time_ranges_condensed = lw.load_json("../data/json/time-ranges-condensed.json")

relevant_bone_types = list(element_grp.groups.keys())
bone_types_dict = dict()

for bone_type in relevant_bone_types:
      ''' 'PERIOD' updaten und alle Spalten entfernen, die nicht benötigt werden '''
      element_dataframe = dataops.update_period(element_grp.get_group(bone_type), time_ranges_condensed)

      ''' save the data for each type to files for better overview '''
      path = "../data/csv/elements/" + bone_type.lower().replace(" ","-")
      #lw.save_csv(element_dataframe, path + ".csv")

      ''' count the amount of different bones for this type '''
      ctr = len(dataops.count_amounts_in_column(element_dataframe, "BONEID"))

      ''' count the amount of periods for this type '''
      periods = len(dataops.count_amounts_in_column(element_dataframe, "PERIOD"))

      bone_types_dict.update(dataops.bone_type_dict(bone_type, element_dataframe, ctr, periods))

Die Anzahl der Funde und Messungen pro Knochentyp und auch pro Periode sollen als Barchart abgebildet werden.
Zur einfacheren Weiterverarbeitung werden diese Daten daher in passende Dataframes sortiert und abgespeichert:

In [19]:
df_measures_findings, df_findings_periods = sph.create_dataframe_for_barchart(bone_types_dict)

#lw.save_csv(df_measures_findings, "../data/csv/amt-measures-findings.csv")

Astragalus : 81 Fund(e) und 229 Messungen. Datierungen für 3 Perioden.
Calcaneum : 33 Fund(e) und 46 Messungen. Datierungen für 3 Perioden.
Femur : 51 Fund(e) und 108 Messungen. Datierungen für 3 Perioden.
First phalanx : 134 Fund(e) und 540 Messungen. Datierungen für 3 Perioden.
Humerus : 97 Fund(e) und 208 Messungen. Datierungen für 3 Perioden.
Mandible has not enough data.
Metacarpal : 112 Fund(e) und 455 Messungen. Datierungen für 3 Perioden.
Metatarsal : 146 Fund(e) und 537 Messungen. Datierungen für 4 Perioden.
Pelvis : 34 Fund(e) und 58 Messungen. Datierungen für 3 Perioden.
Radius : 125 Fund(e) und 367 Messungen. Datierungen für 3 Perioden.
Scapula : 83 Fund(e) und 198 Messungen. Datierungen für 3 Perioden.
Second phalanx has not enough data.
Tibia : 135 Fund(e) und 335 Messungen. Datierungen für 4 Perioden.
Ulna has not enough data.


#### Aussortiern
Die Einträge mit weniger als 10 Funden und/oder weniger als 2 Perioden werden in der weiteren Betrachtung außer Acht gelassen, da sie nicht ausreichend Daten für einen Vergleich liefern. Dies betrifft die Einträge für "Mandible", "Second phalanx" und "Ulna".
Die Anzahl an Knochentypen reduziert sich damit von 14 auf 11.

## Measurements
Als Nächstes gehen wir das Attribut `MEASTYPE` an.
In diesem Datensatz haben wir es mit einer Vielzahl unterschiedlicher Maße zu tun.
Zum einen wurden viele verschiedene Knochentypen vermessen, zum anderen wurden die einzelnen Knochentypen auch noch auf verschiedene Weisen vermessen, was durch den jeweiligen Code unter dem Attribut `MEASTYPE` angezeigt wird.
Das bedeutet, dass selbst die Maße für den gleichen Knochentyp nicht unbedingt vergleichbar sind, weil z.B. manche Maße die Länge des Knochens, andere die Breite an einer bestimmten Stelle und wieder andere den Umfang angeben.
Bei der Auswertung muss also nicht nur nach Knochentyp unterschieden werden, sondern innerhalb des Knochentyps auch noch mal nach `MEASTYPE`, um die Maße auch wirklich vergleichbar zu halten.
Erst dann kann eine Entwicklung über die Zeit betrachtet werden.

#### Problem
Es wird jedoch schnell offensichtlich, dass die vergleichbare Datenmenge plötzlich sehr klein wird und kaum noch für eine aussagekräftige Statistik genutzt werden kann.
Der Versuch, statt aller verfügbarer Pferdeknochen beispielsweise alle Knochen desselben Typs einer Tierart zu untersuchen, die viel mehr Funde vorweisen kann, scheitert leider daran, dass die Datenbank des ABMAP-Projekts momentan nicht verfügbar ist (Stand 31.3.2023). Wir müssen also mit den - vergleichsweise wenigen - Pferdeknochen arbeiten.

Zunächst wollen wir also herausfinden, mit welchen und wie vielen verschiedenen Mess-Dimensionen wir es hier zu tun haben und was sie bedeuten - ggf. kann man sie zusammenfassen. Dazu zählen wir zuerst, wie viele verschiedene Dimensionen pro Knochentyp vorliegen:

In [20]:
element_grp = df_horse.groupby(['ELEMENT'])
df_element_meastype = element_grp['MEASTYPE'].value_counts().to_frame(name='COUNT').reset_index()
df_element_meastype

,ELEMENT,MEASTYPE,COUNT
0,Astragalus,GB,65
1,Astragalus,GH,60
2,Astragalus,BFd,51
3,Astragalus,LmT,51
4,Astragalus,Lm,1
5,Calcaneum,GB,24
6,Calcaneum,GL,22
7,Femur,DC,24
8,Femur,SD,24
9,Femur,Bd,23


Auf der Website des ABMAP gibt es eine Tabelle mit Erklärungen für jeden `MEASTYPE`-Code.
Diese Tabelle speichern wir und fügen anschließend die entsprechenden Erklärungen für jede Dimensionen in die Tabelle ein, um eine bessere Übersicht zu bekommen, welche Dimensionen sich ggf. zusammenfassen lassen.
Diesen Dataframe speichern wir als Tabelle ab, um ihn besser lesen zu können.

In [21]:
df_meastypes = lw.load_csv("../data/csv/meastype.csv")
df_element_meastype['EXPLANATION'] = pd.Series(dtype='string')
for code, explanation in zip(df_meastypes.Code, df_meastypes.Measurement):
      filt = (df_element_meastype['MEASTYPE'] == code)
      df_element_meastype.loc[filt, 'EXPLANATION'] = explanation

#lw.save_csv(df_element_meastype, "../data/csv/bone-elements-w-meastype.csv")

Anscheinend ist das Zusammenfassen der Dimensionen aber nicht sinnvoll (s. Levine1981).
Also suchen wir uns stattdessen für jeden Knochentyp die vier häufigsten Dimensionen heraus und werten diese statistisch aus, um anschließend einen Vergleich über die vier Zeitperioden ziehen zu können.

In [22]:
relevant_bone_types = lw.load_json("../data/json/relevant-bone-types.json")

for bone_type in relevant_bone_types:
      root = "../data/csv/elements/"
      name = bone_type.lower().replace(" ","-")
      df_element = lw.load_csv(root + name + ".csv")
      dimensions_dict = dataops.dimensions_dict(df_element.groupby('MEASTYPE'))

      df_dimensions = pd.DataFrame(columns=['BONEID', 'MEASURE', 'MEASTYPE', 'PERIOD', 'RANGE'])

      ''' now find the 4 most common meastypes '''
      dimensions_dict = dataops.trim_dictionary(dimensions_dict, 4)
      for dimension in dimensions_dict:
            df_dimensions = pd.concat([df_dimensions, dimensions_dict[dimension]["df"]])

      ''' then save those to csv for further processing '''
      #lw.save_csv(df_dimensions, root + "meastypes/" + name + "_meastypes.csv")